In [1]:
import numpy as np
import pandas as pd
import yaml
import sys
import os
import pickle
from glob import glob
from pathlib import Path
import pickle
import warnings
warnings.filterwarnings('ignore')

sys.path.append('/kaggle/src')
from utils.xgb import fit_xgb
from utils.metric import compute_comptetition_metric
from utils.postprocess import post_process
from utils.set_seed import seed_base
from feature_engineering.stage1 import generate_1st_stage_features

PACKAGE_DIR = Path("/kaggle/src")
CFG = yaml.safe_load(open(PACKAGE_DIR / "config.yaml", "r"))
print(CFG["1st_stage"]["execution"]["exp_id"])

CFG["output_dir"] = f"/kaggle/output/{CFG['1st_stage']['execution']['exp_id']}"
!rm -r {CFG["output_dir"]}
os.makedirs(CFG["output_dir"], exist_ok=True)

seed_base(CFG["env"]["seed"])

2023-10-01 16:12:56.381531: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-01 16:12:56.557347: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-01 16:12:57.092216: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda/lib:/usr/local/lib/x86_64-linux-gnu:/usr/local/nvidia/lib:/u

exp_050
rm: cannot remove '/kaggle/output/exp_050': No such file or directory


In [2]:
# 特徴生成
files = glob(f"{CFG['dataset']['step_csv_dir']}/*.csv")
train, features = generate_1st_stage_features(files)

# cv splitとマージ
cv_split = pd.read_csv(CFG['dataset']['cv_split_path'])
train["fold"] = train["series_id"].map(cv_split.set_index("series_id")["fold"])
display(train.head(5))
print(train.shape)

generate features: 100%|██████████| 277/277 [07:01<00:00,  1.52s/it]


,anglez_diff_abs,anglez_diff_abs_rolling_max_10,anglez_diff_abs_rolling_max_100,anglez_diff_abs_rolling_max_1000,anglez_diff_abs_rolling_max_50,anglez_diff_abs_rolling_mean_10,anglez_diff_abs_rolling_mean_100,anglez_diff_abs_rolling_mean_1000,anglez_diff_abs_rolling_mean_50,anglez_diff_abs_rolling_median_10,...,enmo_rolling_square_mean_50,enmo_rolling_std_10,enmo_rolling_std_100,enmo_rolling_std_1000,enmo_rolling_std_50,total_seconds,target,step,series_id,fold
0,0.006755,0.045741,NaN,NaN,NaN,0.008228,NaN,NaN,NaN,0.000453,...,NaN,0.000000,NaN,NaN,NaN,40555.0,1,11.0,af91d9a50547,3
1,0.000122,0.000557,NaN,NaN,0.051780,0.000113,NaN,NaN,0.002316,0.000000,...,0.000000,0.000000,NaN,NaN,0.000000,40670.0,1,34.0,af91d9a50547,3
2,0.000726,0.003443,0.5969,NaN,0.400217,0.000802,0.022484,NaN,0.020798,0.000207,...,0.000003,0.000000,0.00146,NaN,0.001436,40785.0,1,57.0,af91d9a50547,3
3,0.085470,0.348026,0.5969,NaN,0.596900,0.082136,0.024780,NaN,0.041755,0.034920,...,0.000004,0.002008,0.00146,NaN,0.002064,40900.0,1,80.0,af91d9a50547,3
4,0.008796,0.051000,0.5969,NaN,0.405178,0.012594,0.028136,NaN,0.034873,0.004185,...,0.000002,0.000000,0.00146,NaN,0.000990,41015.0,1,103.0,af91d9a50547,3


(5563016, 203)


In [3]:
# 学習
trn_oof, models = fit_xgb(
    X=train, 
    y=train["target"], 
    folds=train["fold"].astype(int), 
    features=features.all_features(),
    params=CFG["1st_stage"]["xgboost"], 
    es_rounds=100,
)
train["oof"] = trn_oof
train = train.drop(columns=features.all_features())
train = train.sort_values(["series_id", "step"]).reset_index(drop=True)

# 保存
for i, model in enumerate(models):
    model.save_model(os.path.join(CFG["output_dir"], f'xgb_fold{i}.model'))
with open(os.path.join(CFG["output_dir"], "features.pkl"), "wb") as f:
    pickle.dump(features, f)
train.to_csv(os.path.join(CFG["output_dir"], "oof.csv"), index=False)

  0%|          | 0/5 [00:00<?, ?it/s]

== fold 0 ==
[0]	eval-logloss:0.61247
[100]	eval-logloss:0.09793
[200]	eval-logloss:0.09833
[229]	eval-logloss:0.09859


 20%|██        | 1/5 [00:35<02:20, 35.04s/it]

== fold 1 ==
[0]	eval-logloss:0.61227
[100]	eval-logloss:0.08199
[200]	eval-logloss:0.08130
[275]	eval-logloss:0.08200


 40%|████      | 2/5 [01:10<01:45, 35.32s/it]

...

 60%|██████    | 3/5 [01:41<01:06, 33.16s/it]

== fold 3 ==
[0]	eval-logloss:0.61297
[100]	eval-logloss:0.09306
[200]	eval-logloss:0.09221
[300]	eval-logloss:0.09275
[313]	eval-logloss:0.09297


 80%|████████  | 4/5 [02:16<00:34, 34.19s/it]

== fold 4 ==
[0]	eval-logloss:0.61349
[100]	eval-logloss:0.10523
[200]	eval-logloss:0.10452
[300]	eval-logloss:0.10395
[400]	eval-logloss:0.10416
[407]	eval-logloss:0.10432


100%|██████████| 5/5 [02:58<00:00, 35.65s/it]


In [4]:
# 後処理
sub = post_process(train)
sub.to_csv(os.path.join(CFG["output_dir"], "submission.csv"), index=False)

# スコア計算
labels = pd.read_csv(f"{CFG['dataset']['competition_dir']}/train_events.csv").dropna()
score, ap_table = compute_comptetition_metric(labels, sub)
print(f"score: {score:.4f}")
display(ap_table)

post process: 100%|██████████| 277/277 [00:49<00:00,  5.56it/s]


score: 0.5044


event   tolerance
onset   12           0.004411
        36           0.046357
        60           0.152982
        90           0.350572
        120          0.512056
        150          0.612113
        180          0.670808
        240          0.727442
        300          0.760642
        360          0.775266
wakeup  12           0.023016
        36           0.176065
        60           0.355648
        90           0.506840
        120          0.607997
        150          0.672585
        180          0.718312
        240          0.778937
        300          0.805807
        360          0.829373
dtype: float64

In [5]:
# 次の候補を決定
next_cand_size = 0
count = 0
next_dict = {}
for series_id, train_df in train.groupby("series_id"):
    train_df = train_df[(train_df["oof"] >= 0.1) & (train_df["oof"] <= 0.9)]
    sub_df = sub[(sub["series_id"] == series_id)]
    label_df = labels[labels["series_id"] == series_id]
    pred_steps = train_df["step"].values
    sub_steps = sub_df["step"].values
    label_steps = label_df["step"].values

    if len(train_df) == 0:
        continue
    next_cand = np.zeros(int(max(max(pred_steps if len(pred_steps) > 0 else [0]), max(sub_steps if len(sub_steps) > 0 else [0]))) + CFG["feature"]["agg_freq"])
    for sub_step in sub_steps:
        next_cand[int(sub_step - CFG["feature"]["agg_freq"] * 10): int(sub_step + CFG["feature"]["agg_freq"] * 10)] = 1
    for pred_step in pred_steps:
        next_cand[int(pred_step - CFG["feature"]["agg_freq"] * 10): int(pred_step + CFG["feature"]["agg_freq"] * 10)] = 1
    next_cand_size += np.sum(next_cand)
    next_dict[series_id] = np.where(next_cand)[0]

    for label_step in label_steps:
        if label_step < next_cand.shape[0]:
            count += next_cand[int(label_step)]
    
recall = count / len(labels)
print(f"recall: {recall:.4f}")
print(f"next_cand_size: {next_cand_size}")

with open(f"{CFG['output_dir']}/next_cands.pkl", "wb") as f:
    pickle.dump(next_dict, f)

recall: 0.9722
next_cand_size: 20842937.0
